In [10]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [11]:
torch.manual_seed(2)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 150 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 11
piter = 5
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [12]:
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', state_dim_SCON + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_CON)
print(obs_error_CON)

tensor([[4.5660e+01, 4.5647e+01, 4.5650e+01, 4.5643e+01, 4.5638e+01, 4.5640e+01,
         4.5626e+01, 4.5630e+01, 4.5618e+01, 4.5617e+01, 4.5615e+01, 4.5605e+01,
         4.5609e+01, 4.5595e+01, 4.5598e+01, 4.5591e+01],
        [7.1469e-02, 7.3700e-02, 7.3289e-02, 7.4423e-02, 7.5230e-02, 7.4875e-02,
         7.6935e-02, 7.6316e-02, 7.8034e-02, 7.7924e-02, 7.8087e-02, 7.9339e-02,
         7.8534e-02, 8.0440e-02, 7.9792e-02, 8.0558e-02],
        [7.1469e-01, 7.1372e-01, 7.1409e-01, 7.1363e-01, 7.1332e-01, 7.1362e-01,
         7.1266e-01, 7.1317e-01, 7.1240e-01, 7.1253e-01, 7.1251e-01, 7.1189e-01,
         7.1246e-01, 7.1149e-01, 7.1193e-01, 7.1158e-01],
        [1.1000e-03, 1.8381e-03, 4.6086e-04, 3.0443e-03, 4.7108e-04, 1.8966e-03,
         1.1670e-03, 7.0596e-04, 2.7906e-03, 4.2941e-04, 2.8298e-03, 7.3398e-04,
         1.2239e-03, 2.0338e-03, 5.1562e-04, 3.3495e-03]])
tensor([[4.5624e+00, 7.6809e-03, 7.1285e-02, 1.5369e-04]])


In [13]:
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB)
print(obs_error_AWB)

tensor([[5.3602e+01, 5.3588e+01, 5.3591e+01, 5.3584e+01, 5.3578e+01, 5.3579e+01,
         5.3564e+01, 5.3569e+01, 5.3560e+01, 5.3559e+01, 5.3558e+01, 5.3545e+01,
         5.3551e+01, 5.3536e+01, 5.3538e+01, 5.3531e+01],
        [1.1421e-01, 1.1832e-01, 1.1810e-01, 1.1999e-01, 1.2125e-01, 1.2086e-01,
         1.2401e-01, 1.2294e-01, 1.2498e-01, 1.2491e-01, 1.2482e-01, 1.2692e-01,
         1.2544e-01, 1.2826e-01, 1.2713e-01, 1.2800e-01],
        [6.7073e-01, 6.7239e-01, 6.7187e-01, 6.7274e-01, 6.7348e-01, 6.7354e-01,
         6.7549e-01, 6.7470e-01, 6.7601e-01, 6.7623e-01, 6.7639e-01, 6.7818e-01,
         6.7750e-01, 6.7964e-01, 6.7946e-01, 6.8049e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3419e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6757e-03, 5.7473e-04, 2.4797e-03, 5.9680e-04, 1.7478e-03,
         1.2328e-03, 8.2910e-04, 2.4061

In [14]:
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB_ECA)
print(obs_error_AWB_ECA)

tensor([[5.3606e+01, 5.3592e+01, 5.3594e+01, 5.3586e+01, 5.3579e+01, 5.3581e+01,
         5.3567e+01, 5.3572e+01, 5.3559e+01, 5.3558e+01, 5.3555e+01, 5.3545e+01,
         5.3550e+01, 5.3533e+01, 5.3538e+01, 5.3528e+01],
        [1.9081e-01, 1.9506e-01, 1.9527e-01, 1.9779e-01, 1.9962e-01, 1.9968e-01,
         2.0286e-01, 2.0196e-01, 2.0493e-01, 2.0501e-01, 2.0569e-01, 2.0746e-01,
         2.0642e-01, 2.0993e-01, 2.0873e-01, 2.1039e-01],
        [6.7073e-01, 6.7236e-01, 6.7196e-01, 6.7284e-01, 6.7358e-01, 6.7335e-01,
         6.7505e-01, 6.7448e-01, 6.7616e-01, 6.7630e-01, 6.7668e-01, 6.7806e-01,
         6.7749e-01, 6.7978e-01, 6.7921e-01, 6.8067e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3418e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6543e-03, 5.6910e-04, 2.4475e-03, 5.8824e-04, 1.7276e-03,
         1.2063e-03, 8.1481e-04, 2.3599

In [15]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

In [16]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [17]:
#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [18]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [19]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [20]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_no_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    obs_model_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFLOW(DEVICE, BATCH_SIZE, obs_model_no_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                #l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                #l1_norm.backward()
                l2_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_lik.mean() - obs_model_CO2(x_with_CO2) + log_prob.mean() #obs_model_CO2(x_with_CO2) is obs log likelihood.
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                neg_ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [21]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.503454. Best norm loss value is: 1735.0345458984375.

x with CO2 means across time = tensor([[0.9018, 0.7048, 0.6691, 0.0029],
        [0.8806, 0.7082, 0.6642, 0.0030]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0500e-01, 9.6841e-01, 6.6545e-01, 3.4796e-03],
         [6.3360e-01, 5.9831e-01, 6.7865e-01, 2.2781e-03],
         ...,
         [6.9264e-01, 6.2056e-01, 5.5705e-01, 4.8133e-03],
         [6.9130e-01, 5.4380e-01, 6.3485e-01, 4.2789e-03],
         [1.0544e+00, 9.8098e-01, 8.4000e-01, 7.6251e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0500e-01, 7.1802e-01, 5.3653e-01, 2.5892e-03],
         [7.3629e-01, 9.0377e-01, 8.6091e-01, 3.4132e-03],
         ...,
         [2.2301e+00, 8.0130e-01, 4.7395e-01, 6.2161e-03],
         [7.8987e-01, 7.1217e-01, 8.7588e-01, 5.6105e-03],
         [4.5167e-01, 7.3495e-01, 3.9585e-01, 5.6243e-03


Train Diffusion:  91%|█████████ | 10/11 [00:08<00:00,  1.24it/s]

Moving average ELBO loss at 10 iterations is: 5.0000000000000066e+19. Best ELBO loss value is: 101257.203125.

x with CO2 means across time = tensor([[3.9836e+00, 4.5496e-01, 2.3343e-01, 1.9611e-03],
        [3.9016e+00, 4.5175e-01, 2.3217e-01, 1.9487e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [1.4100e+00, 4.4540e+00, 2.7983e-01, 1.5584e-02],
         [8.0810e-01, 3.9318e-01, 2.7812e-01, 1.4801e-03],
         ...,
         [3.6730e-01, 2.6524e-01, 1.9221e-01, 2.0476e-03],
         [5.2260e+00, 2.8823e-01, 2.4490e-01, 2.5106e-03],
         [6.5504e+00, 3.8532e-01, 4.2243e-01, 3.3641e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [9.5268e-01, 3.6361e-01, 3.0249e-01, 1.3260e-03],
         [3.4631e+00, 4.2850e+00, 3.1724e-01, 1.5637e-02],
         ...,
         [7.1505e+00, 3.3728e-01, 1.3777e-01, 2.9416e-03],
         [1.7742e+00, 3.5435e-01, 1.8519e-01, 2.7936e-03],
         [1.5860


Train Diffusion: 100%|██████████| 11/11 [00:09<00:00,  1.20it/s]


In [22]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.960657. Best norm loss value is: 1739.6065673828125.

x with CO2 means across time = tensor([[0.7711, 0.6480, 0.6835, 0.0027],
        [0.7723, 0.6575, 0.6856, 0.0028]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [5.2013e-01, 4.4403e-01, 6.7537e-01, 1.6505e-03],
         [5.3253e-01, 6.2790e-01, 7.1086e-01, 2.3881e-03],
         ...,
         [5.4625e-01, 6.6224e-01, 6.9197e-01, 5.1548e-03],
         [9.2636e-01, 7.0183e-01, 6.0733e-01, 5.4610e-03],
         [6.5080e-01, 4.9540e-01, 5.1354e-01, 3.8840e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [2.2908e+00, 5.1948e-01, 6.0545e-01, 1.9352e-03],
         [6.0267e-01, 2.2338e-01, 5.2195e-01, 8.9637e-04],
         ...,
         [7.3852e-01, 6.4371e-01, 6.6433e-01, 5.0197e-03],
         [6.7684e-01, 6.6689e-01, 7.8426e-01, 5.2395e-03],
         [1.0545e+00, 6.9936e-01, 5.3559e-01, 5.4341e-03


Train Diffusion:  91%|█████████ | 10/11 [00:07<00:00,  1.29it/s]

Moving average ELBO loss at 10 iterations is: 5.00000000000002e+19. Best ELBO loss value is: 96780.9765625.

x with CO2 means across time = tensor([[0.5392, 0.6288, 1.7498, 0.0028],
        [0.5404, 0.6391, 1.7076, 0.0029]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.9297e-01, 2.5792e-01, 1.1983e+00, 1.0745e-03],
         [2.6943e-01, 2.5867e-01, 1.0960e+00, 1.1012e-03],
         ...,
         [9.1526e-01, 4.8127e-01, 3.5239e+00, 4.6724e-03],
         [6.5664e-01, 8.8965e-01, 1.4737e+00, 7.1034e-03],
         [4.7572e-01, 1.1404e+00, 1.0788e+00, 8.8533e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.9297e-01, 5.6595e-01, 1.0339e+00, 2.1248e-03],
         [3.2075e-01, 5.3037e-01, 1.1281e+00, 2.0912e-03],
         ...,
         [3.8568e-01, 1.5008e+00, 6.7102e-01, 1.1368e-02],
         [4.7957e-01, 1.6255e+00, 1.9787e+00, 1.2726e-02],
         [5.5251e-01, 1.4184e+00, 1.0300e+00, 1.0916e-02]


Train Diffusion: 100%|██████████| 11/11 [00:08<00:00,  1.37it/s]


In [23]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000205.81626. Best norm loss value is: 2058.16259765625.

x with CO2 means across time = tensor([[0.5008, 0.6816, 0.6848, 0.7171, 0.0037],
        [0.5043, 0.6840, 0.6889, 0.7140, 0.0037]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3321e-01, 4.3474e-01, 1.0880e+00, 6.9862e-01, 3.1244e-03],
         [6.9464e-01, 4.6400e-01, 5.7595e-01, 5.4932e-01, 2.2730e-03],
         ...,
         [4.8385e-01, 4.5988e-01, 6.2850e-01, 1.1344e+00, 5.0452e-03],
         [3.6829e-01, 8.0138e-01, 5.8095e-01, 1.4550e+00, 7.1457e-03],
         [4.7654e-01, 1.1039e+00, 6.5183e-01, 2.5571e-01, 9.5578e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3321e-01, 5.4584e-01, 7.4226e-01, 8.7741e-01, 2.9507e-03],
         [7.2044e-01, 5.4362e-01, 7.6255e-01, 6.3312e-01, 3.1188e-03],
         ...,
         [3.5590e-01, 6.9968e-01, 7.6734e-01, 4.4044e


Train Diffusion:  91%|█████████ | 10/11 [00:09<00:00,  1.11it/s]

Moving average ELBO loss at 10 iterations is: 5.000000000000133e+19. Best ELBO loss value is: 374353.125.

x with CO2 means across time = tensor([[3.2969e+00, 2.0246e-01, 3.2546e-01, 7.8089e-01, 8.2119e-04],
        [3.3380e+00, 2.0030e-01, 3.2353e-01, 7.7182e-01, 8.1522e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [3.4579e-01, 2.1465e+00, 2.3640e-01, 9.5763e-01, 2.4995e-03],
         [5.1904e-01, 1.5889e+00, 2.3710e-01, 3.2402e-01, 2.3130e-03],
         ...,
         [6.0364e+00, 3.0987e-01, 3.4694e-01, 7.6485e-01, 2.3654e-03],
         [4.1931e+00, 2.9317e-01, 5.0825e-01, 2.2985e+00, 3.0245e-03],
         [1.4885e+00, 2.4217e-01, 3.2519e-01, 4.5420e+00, 1.8392e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [3.4579e-01, 5.5896e-01, 2.4748e-01, 3.7267e-01, 1.2760e-03],
         [4.2377e-01, 7.6520e-01, 3.0270e-01, 4.8831e-01, 1.9435e-03],
         ...,
      


Train Diffusion: 100%|██████████| 11/11 [00:10<00:00,  1.08it/s]


In [24]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000203.701075. Best norm loss value is: 2037.0107421875.

x with CO2 means across time = tensor([[1.0512, 0.8083, 0.7275, 0.6679, 0.0044],
        [1.0491, 0.8183, 0.7335, 0.7316, 0.0045]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.0530e+00, 1.0481e+00, 7.7885e-01, 1.1780e+00, 4.8119e-03],
         [1.0485e+00, 8.5329e-01, 7.4345e-01, 5.3931e-01, 4.2387e-03],
         ...,
         [8.5906e-01, 8.3141e-01, 6.8876e-01, 5.1429e-01, 8.2831e-03],
         [1.0337e+00, 7.8649e-01, 6.2817e-01, 4.4090e-01, 7.4814e-03],
         [8.0493e-01, 7.8113e-01, 1.2550e+00, 5.3589e-01, 1.1819e-02]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.0530e+00, 9.5503e-01, 7.6302e-01, 5.0270e-01, 4.4675e-03],
         [1.0203e+00, 8.9388e-01, 8.0379e-01, 5.0645e-01, 4.6211e-03],
         ...,
         [8.4545e-01, 8.6610e-01, 4.9489e-01, 1.5762e


Train Diffusion:  91%|█████████ | 10/11 [00:09<00:00,  1.15it/s]

Moving average ELBO loss at 10 iterations is: 5.000000000000855e+19. Best ELBO loss value is: 8949225.0.

x with CO2 means across time = tensor([[3.3222e+00, 4.3131e-01, 6.9241e-01, 1.3144e+00, 2.4548e-03],
        [3.2819e+00, 4.3005e-01, 6.7814e-01, 1.2706e+00, 2.4850e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [6.3872e-01, 3.6369e+00, 2.4111e-01, 1.1462e+00, 2.9955e-03],
         [2.3157e-01, 7.6413e-01, 2.5328e-01, 6.9009e-01, 1.6645e-03],
         ...,
         [4.7879e+00, 6.0066e-01, 6.1177e-01, 5.0008e-01, 6.0546e-03],
         [1.7795e+00, 4.0945e-01, 8.2448e-01, 2.6958e+00, 5.5257e-03],
         [9.4208e-01, 3.9789e-01, 1.1662e+00, 5.4253e-01, 6.6243e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [6.3873e-01, 1.2952e+00, 8.2293e-01, 8.6826e-01, 5.6960e-03],
         [5.5632e-01, 8.7010e-01, 8.2893e-01, 1.1136e+00, 4.6365e-03],
         ...,
       


Train Diffusion: 100%|██████████| 11/11 [00:09<00:00,  1.11it/s]


In [25]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000204.967505. Best norm loss value is: 2049.675048828125.

x with CO2 means across time = tensor([[0.7278, 0.7266, 0.7496, 0.7743, 0.0042],
        [0.7162, 0.7266, 0.7433, 0.7781, 0.0042]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [6.2883e-01, 8.5254e-01, 7.7909e-01, 7.0263e-01, 4.2058e-03],
         [7.2580e-01, 7.8191e-01, 7.8786e-01, 7.1436e-01, 4.1593e-03],
         ...,
         [7.4278e-01, 7.8743e-01, 8.8683e-01, 7.5617e-01, 9.5190e-03],
         [6.4035e-01, 7.4084e-01, 7.5891e-01, 7.5171e-01, 8.2240e-03],
         [8.6943e-01, 6.3300e-01, 8.6118e-01, 7.7401e-01, 8.0004e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [6.2883e-01, 8.2278e-01, 5.9517e-01, 7.8432e-01, 3.3748e-03],
         [7.0661e-01, 8.0448e-01, 7.6987e-01, 1.4965e+00, 4.1743e-03],
         ...,
         [6.5060e-01, 6.6929e-01, 7.1882e-01, 1.007


Train Diffusion:  91%|█████████ | 10/11 [00:09<00:00,  1.07it/s]

Moving average ELBO loss at 10 iterations is: 5.0000000000001606e+19. Best ELBO loss value is: 898613.375.

x with CO2 means across time = tensor([[0.8893, 0.6261, 0.8039, 1.2455, 0.0038],
        [0.8967, 0.6258, 0.7971, 1.1827, 0.0037]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [6.8114e-01, 4.7031e-01, 4.9920e-01, 1.3398e+00, 1.9864e-03],
         [7.9444e-01, 1.3432e-01, 5.0084e-01, 3.5553e-01, 7.1384e-04],
         ...,
         [1.1553e+00, 3.9707e-01, 6.5498e-01, 8.9795e-01, 4.6201e-03],
         [6.8889e-01, 7.0871e-01, 9.4569e-01, 6.1952e-01, 9.2325e-03],
         [7.6324e-01, 5.1711e-01, 1.1901e+00, 2.6114e+00, 8.3212e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [6.8114e-01, 3.5768e-01, 6.7871e-01, 1.2484e+00, 1.9865e-03],
         [9.7784e-01, 2.0889e-01, 5.9037e-01, 1.8614e+00, 1.1892e-03],
         ...,
         [6.0746e-01, 9.6582e-01, 9.4906e-01, 2.3896


Train Diffusion: 100%|██████████| 11/11 [00:10<00:00,  1.08it/s]


In [26]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000205.154491. Best norm loss value is: 2051.544921875.

x with CO2 means across time = tensor([[0.6652, 0.6404, 0.6544, 0.6427, 0.0035],
        [0.6771, 0.6330, 0.6543, 0.6424, 0.0034]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [5.8845e-01, 6.9575e-01, 5.7162e-01, 7.1231e-01, 2.9229e-03],
         [3.1536e-01, 4.2306e-01, 1.1967e+00, 8.1659e-01, 3.3530e-03],
         ...,
         [6.5951e-01, 5.1156e-01, 6.0266e-01, 6.1256e-01, 5.3157e-03],
         [6.1540e-01, 1.0991e+00, 5.6553e-01, 7.1014e-01, 8.5296e-03],
         [3.9723e-01, 8.1369e-01, 6.6245e-01, 5.3483e-01, 7.9686e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.1560e-01, 6.5541e-01, 4.7142e-01, 6.3648e-01, 2.4208e-03],
         [3.3602e+00, 4.2167e-01, 7.0838e-01, 5.6401e-01, 2.4331e-03],
         ...,
         [8.2287e-01, 6.3645e-01, 7.1959e-01, 6.9419e-


Train Diffusion:  91%|█████████ | 10/11 [00:09<00:00,  1.09it/s]

Moving average ELBO loss at 10 iterations is: 5.000000000000086e+19. Best ELBO loss value is: 986653.8125.

x with CO2 means across time = tensor([[3.4244e+00, 4.4740e-01, 3.9032e-01, 3.6381e-01, 1.7442e-03],
        [3.2444e+00, 4.4414e-01, 3.9535e-01, 3.5982e-01, 1.7582e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.3430e-01, 8.3759e-01, 3.9135e-01, 5.7443e-01, 2.4506e-03],
         [8.0855e+00, 4.6370e-01, 6.6229e-01, 1.9452e-01, 2.5063e-03],
         ...,
         [4.4428e+00, 5.2573e-01, 4.5420e-01, 4.1476e-01, 4.3965e-03],
         [4.5661e+00, 4.2905e-01, 2.8863e-01, 3.2638e-01, 2.6361e-03],
         [8.1634e-01, 2.1256e-01, 3.6774e-01, 6.6947e-01, 1.8106e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.5280e+00, 5.5499e-01, 4.1785e-01, 2.6266e-01, 1.9588e-03],
         [1.8243e-01, 7.8638e-01, 1.8714e-01, 5.1569e-01, 1.2938e-03],
         ...,
     


Train Diffusion: 100%|██████████| 11/11 [00:10<00:00,  1.02it/s]
